In [2]:
import requests
import json


In [4]:
requests.delete("http://localhost:8983/solr/tmdb/schema/model-store/test")

<Response [200]>

In [5]:
requests.delete("http://localhost:8983/solr/tmdb/schema/feature-store/test")

<Response [200]>

In [6]:
feature_set = [
    {
      "name" : "title_bm25",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "title:(${keywords})"
      }
    },
    {
      "name" : "overview_bm25",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    }
]

requests.put('http://localhost:8983/solr/tmdb/schema/feature-store', 
             json=feature_set)

<Response [200]>

In [7]:
resp = requests.get('http://localhost:8983/solr/tmdb/schema/feature-store/test')
resp.json()

{'responseHeader': {'status': 0, 'QTime': 11},
 'features': [{'name': 'title_bm25',
   'class': 'org.apache.solr.ltr.feature.SolrFeature',
   'params': {'q': 'title:(${keywords})'},
   'store': 'test'},
  {'name': 'overview_bm25',
   'class': 'org.apache.solr.ltr.feature.SolrFeature',
   'params': {'q': 'overview:(${keywords})'},
   'store': 'test'}]}

In [8]:
params = {
    "fl": "id,title,[features store=test efi.keywords=\"rambo\"]",
    'q': "id:7555 OR id:1370 id:1369 OR id:1368",
    'rows': 10,
    'wt': 'json'  
}

resp = requests.post('http://localhost:8983/solr/tmdb/select', data=params).json()
resp

{'responseHeader': {'status': 0,
  'QTime': 885,
  'params': {'q': 'id:7555 OR id:1370 id:1369 OR id:1368',
   'fl': 'id,title,[features store=test efi.keywords="rambo"]',
   'rows': '10',
   'wt': 'json'}},
 'response': {'numFound': 4,
  'start': 0,
  'docs': [{'id': '1370',
    'title': ['Rambo III'],
    '[features]': 'title_bm25=10.357876,overview_bm25=11.95039'},
   {'id': '1369',
    'title': ['Rambo: First Blood Part II'],
    '[features]': 'title_bm25=7.010513,overview_bm25=11.220095'},
   {'id': '1368',
    'title': ['First Blood'],
    '[features]': 'title_bm25=0.0,overview_bm25=11.220095'},
   {'id': '7555',
    'title': ['Rambo'],
    '[features]': 'title_bm25=12.318474,overview_bm25=10.573916'}]}}

In [9]:
model = """## LambdaMART
## No. of trees = 10
## No. of leaves = 10
## No. of threshold candidates = 256
## Learning rate = 0.1
## Stop early = 100

<ensemble>
	<tree id="1" weight="0.1">
		<split>
			<feature> 2 </feature>
			<threshold> 10.664251 </threshold>
			<split pos="left">
				<feature> 1 </feature>
				<threshold> 0.0 </threshold>
				<split pos="left">
					<output> -1.8305741548538208 </output>
				</split>
				<split pos="right">
					<feature> 2 </feature>
					<threshold> 9.502127 </threshold>
					<split pos="left">
						<feature> 1 </feature>
						<threshold> 7.0849166 </threshold>
						<split pos="left">
							<output> 0.23645669221878052 </output>
						</split>
						<split pos="right">
							<output> 1.7593677043914795 </output>
						</split>
					</split>
					<split pos="right">
						<output> 1.9719607830047607 </output>
					</split>
				</split>
			</split>
			<split pos="right">
				<feature> 2 </feature>
				<threshold> 0.0 </threshold>
				<split pos="left">
					<output> 1.3728954792022705 </output>
				</split>
				<split pos="right">
					<feature> 2 </feature>
					<threshold> 8.602512 </threshold>
					<split pos="left">
						<feature> 1 </feature>
						<threshold> 0.0 </threshold>
						<split pos="left">
							<feature> 2 </feature>
							<threshold> 13.815164 </threshold>
							<split pos="left">
								<output> 1.9401178359985352 </output>
							</split>
							<split pos="right">
								<output> 1.99532949924469 </output>
							</split>
						</split>
						<split pos="right">
							<feature> 1 </feature>
							<threshold> 11.085816 </threshold>
							<split pos="left">
								<output> 2.0 </output>
							</split>
							<split pos="right">
								<output> 1.99308180809021 </output>
							</split>
						</split>
					</split>
					<split pos="right">
						<output> 1.9870178699493408 </output>
					</split>
				</split>
			</split>
		</split>
	</tree>
</ensemble>
"""

from ltr.helpers.convert import convert

feature_mapping = [{'name': 'title_bm25'}, {'name': 'overview_bm25'}]

solr_model = convert(model, 'test', 'test', feature_mapping)
print(json.dumps(solr_model, indent=2))

{
  "store": "test",
  "name": "test",
  "class": "org.apache.solr.ltr.model.MultipleAdditiveTreesModel",
  "features": [
    {
      "name": "title_bm25"
    },
    {
      "name": "overview_bm25"
    }
  ],
  "params": {
    "trees": [
      {
        "weight": "0.1",
        "root": {
          "feature": "overview_bm25",
          "threshold": "10.664251",
          "left": {
            "feature": "title_bm25",
            "threshold": "0.0",
            "left": {
              "value": "-1.8305741548538208"
            },
            "right": {
              "feature": "overview_bm25",
              "threshold": "9.502127",
              "left": {
                "feature": "title_bm25",
                "threshold": "7.0849166",
                "left": {
                  "value": "0.23645669221878052"
                },
                "right": {
                  "value": "1.7593677043914795"
                }
              },
              "right": {
                "value": "

In [10]:
requests.put('http://localhost:8983/solr/tmdb/schema/model-store',
              json=solr_model)

<Response [200]>

In [12]:
params = {
    'q': '{!ltr model=test efi.keywords=\"rocky\"}',
    'rows': 10,
    'fl': 'title'
}

resp = requests.post('http://localhost:8983/solr/tmdb/select', data=params).json()
resp

{'responseHeader': {'status': 0,
  'QTime': 63,
  'params': {'q': '{!ltr model=test efi.keywords="rocky"}',
   'fl': 'title',
   'rows': '10'}},
 'response': {'numFound': 27760,
  'start': 0,
  'docs': [{'title': ['Oliver!']},
   {'title': ['Bullets Over Broadway']},
   {'title': ['Loose Cannons']},
   {'title': ['Sleeper']},
   {'title': ['Watchmen']},
   {'title': ['Sherlock Holmes']},
   {'title': ['The Hours']},
   {'title': ['Vampire Hunter D: Bloodlust']},
   {'title': ['Biutiful']},
   {'title': ['Melancholia']}]}}

In [14]:
params = {
    'q': 'rambo',
    'rows': 10,
    "rq": "{!ltr model=test reRankDocs=50 efi.keywords=\"rambo\"}",
    'wt': 'json',
    'fl': 'title'
}

resp = requests.post('http://localhost:8983/solr/tmdb/select', data=params).json()
resp

{'responseHeader': {'status': 0,
  'QTime': 36,
  'params': {'q': 'rambo',
   'fl': 'title',
   'rows': '10',
   'wt': 'json',
   'rq': '{!ltr model=test reRankDocs=5 efi.keywords="rambo"}'}},
 'response': {'numFound': 13,
  'start': 0,
  'docs': [{'title': ['Rambo III']},
   {'title': ['Rambo: First Blood Part II']},
   {'title': ['First Blood']},
   {'title': ['Rambo']},
   {'title': ['A Hole in the Soul']},
   {'title': ['In the Line of Duty: The F.B.I. Murders']},
   {'title': ['Which Way to the Front?']},
   {'title': ['Son of Rambow']},
   {'title': ['Zelig']},
   {'title': ['Go for It']}]}}